In [1]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pandas as pd

import re

## Features 

In [2]:
import re

indexes = []
features = []
with open('../homework2_exercise2/data/features', 'r') as f:
    for line in f.readlines():
        index, *feat = line.split(':')
        feat = ''.join(feat)
        feat = re.findall('"([^\"]*)"', feat)
        indexes.append(index)
    
        
        feat += [f.split('_')[0] for f in feat]
        feat = list(set(feat))
        
        features.append(feat)

In [3]:
one_hot = MultiLabelBinarizer()
mhe = one_hot.fit_transform(features)
df = pd.DataFrame(mhe)

In [4]:
df.rename(columns=lambda x: f'feat_{x}', inplace=True)
df = df.T.drop_duplicates().T
df = df.astype(bool)
df['theorem'] = indexes

In [5]:
del mhe
del features
del indexes

# Dataset

## Ones

In [7]:
indexes_train = []
premises = []

ones = []

with open('../homework2_exercise2/data/dependencies_train', 'r') as f:
    for line in f.readlines():
        index, *prem = line.split(':')
        prem = ''.join(prem).split()
        
        indexes_train.append(index)
        premises.append(prem)
        
        for p in prem:
            ones.append((index, p))

In [8]:
ones_df = pd.DataFrame(ones, columns=['theorem', 'premise']).drop_duplicates()

In [9]:
del ones
del premises
del indexes_train

## Zeros

In [17]:
chronology = pd.read_csv('../homework2_exercise2/data/chronology_backup', squeeze=True, names=['chronology'])
def get_available_theorems(name):
    return chronology.loc[:(chronology == name).argmax()-1]

In [18]:
used_premises = ones_df.groupby('theorem')['premise'].unique()

In [19]:
FALSE_EXAMPLES_RATIO = 2

dataset_zeros = []
for theorem, theorem_premises in ones_df.groupby('theorem')['premise']:
    available_theorems = get_available_theorems(theorem)
    zero_premises = np.setdiff1d(available_theorems, theorem_premises)
    
    zeros_chosen = zero_premises[np.random.randint(0, len(zero_premises), FALSE_EXAMPLES_RATIO * theorem_premises.shape[0])]
    dataset_zeros += list(zip(
        [theorem] * len(zeros_chosen),
        zeros_chosen
    ))

/home/piotrek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
zeros_df = pd.DataFrame(dataset_zeros, columns=['theorem', 'premise'])

In [21]:
del dataset_zeros
del used_premises
del chronology

##  Merging

In [22]:
zeros_df['useful'] = 0
ones_df['useful'] = 1

In [23]:
df_train = pd.concat([zeros_df, ones_df])

In [24]:
del zeros_df
del ones_df

In [25]:
df_train.shape

(82299, 3)

In [26]:
df_train = df_train.sample(frac=0.5)

In [27]:
df.shape

(4564, 4872)

In [28]:
df_train = pd.merge(
    df_train, df,
    left_on=['theorem'],
    right_on=['theorem']
)
df_train.head()

,theorem,premise,useful,feat_0,feat_1,feat_2,feat_3,feat_4,feat_6,feat_7,...,feat_10348,feat_10353,feat_10355,feat_10358,feat_10361,feat_10362,feat_10365,feat_10368,feat_10373,feat_10376
0,t12_tops_2,redefinition_k4_subset_1,1,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,t12_tops_2,d6_tops_1,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,t12_tops_2,t23_ordinal1,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,t12_tops_2,dt_k2_subset_1,1,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,t12_tops_2,d6_ordinal1,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [29]:
df_train = pd.merge(
    df_train, df,
    left_on=['premise'],
    right_on=['theorem'],
    suffixes=('_prem', '_thm')
)
df_train.head()

,theorem_prem,premise,useful,feat_0_prem,feat_1_prem,feat_2_prem,feat_3_prem,feat_4_prem,feat_6_prem,feat_7_prem,...,feat_10353_thm,feat_10355_thm,feat_10358_thm,feat_10361_thm,feat_10362_thm,feat_10365_thm,feat_10368_thm,feat_10373_thm,feat_10376_thm,theorem_thm
0,t12_tops_2,redefinition_k4_subset_1,1,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,redefinition_k4_subset_1
1,t2_tex_2,redefinition_k4_subset_1,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,redefinition_k4_subset_1
2,t3_funct_2,redefinition_k4_subset_1,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,redefinition_k4_subset_1
3,t41_subset_1,redefinition_k4_subset_1,1,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,redefinition_k4_subset_1
4,t15_subset_1,redefinition_k4_subset_1,1,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,redefinition_k4_subset_1


In [30]:
df_train['useful'].mean()

0.3343620899149453

In [31]:
import lightgbm as lgb

In [35]:
df_X = df_train.drop(['theorem_prem', 'theorem_thm', 'premise', 'useful'], axis=1)
df_y = df_train['useful']
train_data = lgb.Dataset(df_X, df_y)

In [38]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 1
}
model = lgb.train(
    parameters,
    train_data
)

In [39]:
preds = model.predict(df_X)

In [42]:
preds

array([0.48027697, 0.36631135, 0.17748185, ..., 0.83642001, 0.83480839,
       0.83180305])

In [43]:
from sklearn.metrics import roc_auc_score

In [46]:
roc_auc_score(df_y, preds)

0.8991875557875608

In [165]:
df.columns

Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9',
       ...
       'feat_10370', 'feat_10371', 'feat_10372', 'feat_10373', 'feat_10374',
       'feat_10375', 'feat_10376', 'feat_10377', 'feat_10378', 'feat_theorem'],
      dtype='object', length=10380)

In [125]:
np.random.randint(1, 3, 4)

array([1, 2, 1, 1])

In [78]:
len(ones)

182506

In [102]:
available_theorems('dt_k5_xboole_0')

/home/piotrek/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  


Series([], Name: chronology, dtype: object)

In [95]:
chronology

0                  dt_k5_xboole_0
1                  dt_k4_xboole_0
2         idempotence_k2_xboole_0
3                     d3_xboole_0
4       commutativity_k5_xboole_0
5                     d5_xboole_0
6                     d6_xboole_0
7          antisymmetry_r2_hidden
8       commutativity_k2_xboole_0
9                  dt_k2_xboole_0
10                    t1_xboole_0
11                      t2_tarski
12                    t2_xboole_0
13                   rc1_xboole_0
14                 dt_k3_xboole_0
15              dt_o_0_0_xboole_0
16                    d1_xboole_0
17      commutativity_k3_xboole_0
18        idempotence_k3_xboole_0
19                   fc1_xboole_0
20                   l13_xboole_0
21                    d4_xboole_0
22                   rc2_xboole_0
23                    d2_xboole_0
24           symmetry_r1_xboole_0
25                 dt_k1_xboole_0
26                    d7_xboole_0
27                    t3_xboole_0
28                    t4_xboole_0
29            

In [79]:
pd.Series(ones).drop_duplicates().shape

(27433,)

In [39]:
df = pd.DataFrame(mhe, index=indexes)

In [71]:
train_data = [df.loc[premises_for_ind].sum() for premises_for_ind in premises]
df_train = pd.DataFrame(data=train_data, index=indexes_train, columns=df.columns)

In [103]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,10369,10370,10371,10372,10373,10374,10375,10376,10377,10378
abstractness_v1_orders_2,0,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
abstractness_v1_pre_topc,0,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
abstractness_v3_lattices,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abstractness_v6_waybel_0,0,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
antisymmetry_r2_hidden,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
premises_sums = []
for ind, premises_for_ind in enumerate(premises):
    df_train.iloc[ind] = df.loc[premises_for_ind].sum()

KeyboardInterrupt: 

In [69]:
df_train_sums = pd.DataFrame(premises_sums, index=indexes_train)

In [64]:
df_train.loc['t10_xboole_1'] = df.loc[['d3_tarski','d3_xboole_0']].sum()

ValueError: shape mismatch: value array of shape (4,) could not be broadcast to indexing result of shape (4,10379)

In [65]:
df.loc[['d3_tarski','d3_xboole_0']].sum().shape

(10379,)

In [66]:
df_train.loc['t10_xboole_1']

,0,1,2,3,4,5,6,7,8,9,...,10369,10370,10371,10372,10373,10374,10375,10376,10377,10378
t10_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t10_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t10_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t10_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,t151_funct_1,t58_wellord1,t164_funct_2,t113_tmap_1,t56_tmap_1,t154_funct_1,t76_tops_2,t13_waybel_0,t23_tex_2,t73_tops_2
t3_xboole_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t7_xboole_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t6_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t6_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t9_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,t151_funct_1,t58_wellord1,t164_funct_2,t113_tmap_1,t56_tmap_1,t154_funct_1,t76_tops_2,t13_waybel_0,t23_tex_2,t73_tops_2
t3_xboole_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t7_xboole_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t6_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t6_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
t9_xboole_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df.loc[premises[4]].sum()

0        0
1        0
2        0
3        0
4        3
5        3
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
10349    0
10350    0
10351    0
10352    0
10353    0
10354    0
10355    0
10356    0
10357    0
10358    0
10359    0
10360    0
10361    0
10362    0
10363    0
10364    0
10365    0
10366    0
10367    0
10368    0
10369    0
10370    0
10371    0
10372    0
10373    0
10374    0
10375    0
10376    0
10377    0
10378    0
Length: 10379, dtype: int64

In [31]:
premises

[['d1_xboole_0', 'fc1_xboole_0', 'd4_xboole_0', 'd7_xboole_0'],
 ['d1_xboole_0', 'l13_xboole_0'],
 ['idempotence_k2_xboole_0', 't4_xboole_1'],
 ['t1_boole', 't5_xboole_1', 't4_xboole_1', 'commutativity_k2_xboole_0'],
 ['d10_xboole_0',
  't7_xboole_1',
  't4_xboole_1',
  'commutativity_k2_xboole_0',
  't8_xboole_1'],
 ['t7_xboole_1', 't1_xboole_1', 'commutativity_k2_xboole_0', 't8_xboole_1'],
 ['d3_tarski', 'd3_xboole_0'],
 ['t7_xboole_1', 'commutativity_k2_xboole_0', 't1_xboole_1'],
 ['d10_xboole_0',
  't7_xboole_1',
  't4_xboole_1',
  'commutativity_k2_xboole_0',
  't8_xboole_1'],
 ['d10_xboole_0',
  't7_xboole_1',
  't4_xboole_1',
  'commutativity_k2_xboole_0',
  't9_xboole_1',
  'idempotence_k2_xboole_0'],
 ['d3_tarski', 'd3_xboole_0'],
 ['t7_xboole_1', 't1_xboole_1'],
 ['d10_xboole_0', 't10_xboole_1', 'commutativity_k2_xboole_0', 't1_xboole_1'],
 ['d10_xboole_0',
  't7_xboole_1',
  'commutativity_k2_xboole_0',
  't9_xboole_1',
  'idempotence_k2_xboole_0'],
 ['d10_xboole_0', 't7_xbo

In [28]:
df.columns.duplicated().sum()

0

In [5]:
indexes

['abstractness_v1_orders_2',
 'abstractness_v1_pre_topc',
 'abstractness_v3_lattices',
 'abstractness_v6_waybel_0',
 'antisymmetry_r2_hidden',
 'antisymmetry_r2_xboole_0',
 'cc10_card_3',
 'cc10_funct_1',
 'cc10_lattices',
 'cc10_tdlat_3',
 'cc10_tex_2',
 'cc10_waybel_0',
 'cc10_waybel_1',
 'cc11_card_3',
 'cc11_funct_1',
 'cc11_membered',
 'cc11_tdlat_3',
 'cc11_tex_2',
 'cc11_waybel_0',
 'cc11_yellow_0',
 'cc12_membered',
 'cc12_tdlat_3',
 'cc12_tex_2',
 'cc12_waybel_0',
 'cc12_yellow_0',
 'cc13_membered',
 'cc13_tdlat_3',
 'cc13_tex_2',
 'cc13_waybel_0',
 'cc14_card_3',
 'cc14_membered',
 'cc14_tdlat_3',
 'cc14_tex_2',
 'cc14_waybel_0',
 'cc15_membered',
 'cc15_tdlat_3',
 'cc15_tex_2',
 'cc15_waybel_0',
 'cc16_membered',
 'cc16_tex_2',
 'cc17_membered',
 'cc17_tex_2',
 'cc18_membered',
 'cc18_tex_2',
 'cc19_membered',
 'cc19_tex_2',
 'cc1_card_1',
 'cc1_card_3',
 'cc1_classes2',
 'cc1_compts_1',
 'cc1_connsp_1',
 'cc1_eqrel_1',
 'cc1_filter_0',
 'cc1_finseq_1',
 'cc1_finset_1',
 'cc